In [1]:
from new_parser import construct, match, parser

Generating LALR tables


In [2]:
from utils import SocketReader

In [3]:
try:
    sock.close()
except:
    pass

sock = SocketReader("8.8.8.8", 53, 'udp')

sock.connect()

In [4]:
dns_packet_spec = r"""
16@id[hex]
-1@qr-4@opcode[int]-1@aa-1@tc-1@rd-1@ra-3==0-4@rcode[int]
-16@qcount[int]
-16@ancount[int]
-16@nscount[int]
-16@arcount[int]
-{qcount}...@questions(
    ~\0@question(
        ...8@parts[str]()
    )
    -16@type
    -16@class
)
-{ancount}...@answers(
    (
        2==0b11
        -14@pointer[int]
        |
        ~\0@question(
            ...8@parts()
        )
    )
    -16==1@type
    -16@class
    -32@ttl
    -16@answer[ip]()
)
"""
dns_mappings = {
    'type': [('A', 1), ('CNAME', 5)],
    'class': [('IN', 1)],
}

In [5]:
dns_packet = {
    'id': 0x911f,
    'qr': 0b0,
    'opcode': 0b0000,
    'aa': 0b0,
    'tc': 0b0,
    'rd': 0b1,
    'ra': 0b0,
    'z': 0b000,
    'rcode': 0b0000,
    'qcount': 1,
    'ancount': 0,
    'nscount': 0,
    'arcount': 0,
    'questions': [{
        'question': {
            'parts': [
                "www", "google", "com"
            ],
        },
        'type': 'A',
        'class': 'IN'
    }]
}

In [6]:
question = construct(dns_packet_spec, dns_packet, mappings=dns_mappings)
print(question)

b'\x91\x1f\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x03www\x06google\x03com\x00\x00\x01\x00\x01'


In [7]:
sock.send(question)

In [8]:
resp = sock.read_once()

In [9]:
resp

b'\x91\x1f\x81\x80\x00\x01\x00\x01\x00\x00\x00\x00\x03www\x06google\x03com\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00\x00#\x00\x04\xac\xd9\xa8\xe4'

In [10]:
res, rem = match(dns_packet_spec, resp, mappings=dns_mappings, verbose=True)
res

{'id': 0x911f,
 'qr': 0b1,
 'opcode': 0,
 'aa': 0b0,
 'tc': 0b0,
 'rd': 0b1,
 'ra': 0b1,
 'rcode': 0,
 'qcount': 1,
 'ancount': 1,
 'nscount': 0,
 'arcount': 0,
 'questions': [{'question': {'parts': ['www', 'google', 'com']},
   'type': 'A',
   'class': 'IN'}],
 'answers': [{'pointer': 12,
   'type': 'A',
   'class': 'IN',
   'ttl': 0x00000023,
   'answer': '172.217.168.228'}]}

In [17]:
for answer in res['answers']:
    parts = None
    if 'question' in answer:
        parts = answer['question']['parts']
    else:
        m, _ = match(r"""~\0@question(
            ...8@parts[str]()
        )""", resp[answer['pointer']._value():])
        parts = m['question']['parts']
    ip = '.'.join(i._value() for i in parts)
    print('found {} -> {}'.format(ip, answer['answer']))
        

found www.google.com -> '172.217.168.228'


In [18]:
sock.close()